In [2]:
%matplotlib qt
%load_ext autoreload
%autoreload 2
import mne
import os


subjects_dir = "C:/Users/lukas/mne_data"
subject = "hari"

time_frames = {
    "early": (0.089, 0.109),
    "late": (0.109, 0.129),
    "full": (0, 0.2)
    # "alternative": (0.080, 0.150)
}

In [3]:
# mne.gui.coregistration(subjects_dir=subjects_dir, subject=subject, head_high_res=True)

# Load Data

In [4]:
path = f"{subjects_dir}/{subject}/taskforce_1_bstem.fif"
raw = mne.io.read_raw_fif(path, verbose=0, preload=True)

C:\Users\lukas\AppData\Local\Temp\ipykernel_4356\944929098.py:2: RuntimeWarning: This filename (C:/Users/lukas/mne_data/hari/taskforce_1_bstem.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(path, verbose=0, preload=True)


# Use ICA to remove eye and heart artifacts

In [5]:
import numpy as np
from mne.preprocessing import ICA, create_eog_epochs

raw_filt = raw.copy().filter(1.0, None, method="fir", fir_design="firwin")

ica = ICA(n_components=15, max_iter="auto", random_state=0, verbose=0)
ica.fit(raw_filt, verbose=0)

ica.plot_components(np.arange(15))
ica.plot_sources(raw_filt)
ica.exclude = [0,2,3]
raw = ica.apply(raw, verbose=0)

Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Filter length: 3301 samples (3.301 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 306 out of 306 | elapsed:    6.6s finished


Creating RawArray with float64 data, n_channels=15, n_times=759000
    Range : 90000 ... 848999 =     90.000 ...   848.999 secs
Ready.
Using matplotlib as 2D backend.


In [6]:
raw.info["bads"].append("MEG2212")
picks = mne.pick_types(raw.info, meg=True, exclude="bads")
events = mne.find_events(raw, stim_channel="STI101")
mne.viz.plot_events(events, raw.info["sfreq"])
reject = dict(grad=3000e-13, mag=3e-12)
event_ids = {"auditory_1": 1, "auditory_2": 2}
tmin, tmax = -0.1, 0.5
epochs = mne.Epochs(
    raw,
    events,
    event_ids,
    tmin,
    tmax,
    reject=reject,
    picks=picks,
    baseline=(tmin, 0),
    preload=True
)
raw = raw.pick(picks)
raw.filter(None, 30, method="fir", fir_design="firwin", verbose=0)
epochs.filter(None, 30, method="fir", fir_design="firwin", verbose=0)

346 events found on stim channel STI101
Event IDs: [1 2]
Not setting metadata
346 matching events found
Applying baseline correction (mode: mean)
Created an SSP operator (subspace dimension = 4)
4 projection items activated
Using data from preloaded Raw for 346 events and 601 original time points ...
    Rejecting  epoch based on MAG : ['MEG2641']
    Rejecting  epoch based on MAG : ['MEG1531', 'MEG1541', 'MEG1711']
    Rejecting  epoch based on MAG : ['MEG1221', 'MEG1321', 'MEG1331', 'MEG1431', 'MEG1441', 'MEG2421', 'MEG2521', 'MEG2611', 'MEG2621', 'MEG2631', 'MEG2641']
3 bad epochs dropped


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 305 out of 305 | elapsed:    6.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 104615 out of 104615 | elapsed:   17.7s finished


Number of events,343
Events,auditory_1: 165auditory_2: 178
Time range,-0.100 – 0.500 s
Baseline,-0.100 – 0.000 s


In [7]:
epochs["auditory_1"].average().plot_joint()
epochs["auditory_2"].average().plot_joint()

Projections have already been applied. Setting proj attribute to True.
Removing projector <Projection | axial-10.0-200.0-PCA-01, active : True, n_channels : 102>
Removing projector <Projection | axial-10.0-200.0-PCA-02, active : True, n_channels : 102>
Removing projector <Projection | axial-10.0-200.0-PCA-03, active : True, n_channels : 102>
Removing projector <Projection | axial-10.0-200.0-PCA-04, active : True, n_channels : 102>
Projections have already been applied. Setting proj attribute to True.
Removing projector <Projection | axial-10.0-200.0-PCA-01, active : True, n_channels : 102>
Removing projector <Projection | axial-10.0-200.0-PCA-02, active : True, n_channels : 102>
Removing projector <Projection | axial-10.0-200.0-PCA-03, active : True, n_channels : 102>
Removing projector <Projection | axial-10.0-200.0-PCA-04, active : True, n_channels : 102>


[<Figure size 800x420 with 6 Axes>, <Figure size 800x420 with 6 Axes>]

# Load Forward Model

In [8]:
sampling = "oct6"
trans_fname = os.path.join(subjects_dir, subject, "hari_fit-trans.fif")
# trans_fname = os.path.join(subjects_dir, subject, "hari_fit_manual-trans.fif")
bem_fname = os.path.join(subjects_dir, subject, "bem", "RON026_AKCLEE-5120-5120-5120-bem-sol.fif")
# bem_fname = os.path.join(subjects_dir, subject, "bem", "RON026_AKCLEE-5120-bem-sol.fif")

src = mne.setup_source_space(subject, spacing=sampling, surface='white',
                                        subjects_dir=subjects_dir, add_dist=False,
                                        n_jobs=-1, verbose=0)

fwd = mne.make_forward_solution(epochs.info, trans_fname, src, bem_fname, verbose=0)
fwd = mne.convert_forward_solution(fwd, surf_ori=True, force_fixed=True,
                                            use_cps=True, verbose=0)

n_chans, n_dipoles = fwd["sol"]["data"].shape

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.9s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent work

# Define Solvers

In [9]:
%matplotlib qt
import sys; sys.path.insert(0, r'../../invert')
from invert import Solver
from invert.evaluate import eval_mean_localization_error
from scipy.stats import pearsonr
from scipy.spatial.distance import cdist
from funs import get_mirrored_stc

prep_leadfield = False
n_orders = 0
diffusion_parameter = 0.1
max_iter = 5
stop_crit = 0
n_sources = 2
n_reg_params = 1

src = fwd["src"]
adjacency = mne.spatial_src_adjacency(src, verbose=0).toarray()

# get position of vertices (only the onew that are used)
pos_left = mne.vertex_to_mni(src[0]["vertno"], 0, subject=subject, subjects_dir=subjects_dir, verbose=0)
pos_right = mne.vertex_to_mni(src[1]["vertno"], 1, subject=subject, subjects_dir=subjects_dir, verbose=0)
pos = np.concatenate([pos_left, pos_right], axis=0)

# distance_matrix = cdist(pos, pos)

solver_dicts = [
    {
        "make_args": dict(n=n_sources, k=n_sources, n_orders=n_orders, stop_crit=stop_crit, max_iter=max_iter, diffusion_parameter=diffusion_parameter),
        "solver_name": "RAP-MUSIC",
        "display_name": f"RAP-MUSIC",
        "prep_leadfield": prep_leadfield,
        "clim": {"kind": "value", "lims": [0., 0., 1]},
    },
    # {
    #     "make_args": dict(n=n_sources, k=n_sources, n_orders=n_orders, stop_crit=stop_crit, max_iter=max_iter, diffusion_parameter=diffusion_parameter),
    #     "solver_name": "RAP-MUSIC",
    #     "display_name": f"FLEX-RAP-MUSIC",
    #     "prep_leadfield": prep_leadfield,
    #     "clim": {"kind": "value", "lims": [0., 0., 1]},
    # },
    {
        "make_args": dict(n=n_sources, k=n_sources, n_orders=n_orders, stop_crit=stop_crit, max_iter=max_iter, diffusion_parameter=diffusion_parameter),
        "solver_name": "AP",
        "display_name": f"AP",
        "prep_leadfield": prep_leadfield,
        "clim": {"kind": "value", "lims": [0., 0., 1]},
    },
    # {
    #     "make_args": dict(n=n_sources, k=n_sources, n_orders=n_orders, stop_crit=stop_crit, max_iter=max_iter, diffusion_parameter=diffusion_parameter),
    #     "solver_name": "FLEX-AP",
    #     "display_name": f"FLEX-AP",
    #     "prep_leadfield": prep_leadfield,
    #     "clim": {"kind": "value", "lims": [0., 0., 1]},
    # },
    # {
    #     "make_args": dict(n=n_sources, k=n_sources, n_orders=n_orders, stop_crit=stop_crit, max_iter=max_iter, diffusion_parameter=diffusion_parameter),
    #     "solver_name": "SSM",
    #     "display_name": f"FLEX-SSM",
    #     "prep_leadfield": prep_leadfield,
    #     "clim": {"kind": "value", "lims": [0., 0., 1]},
    # },
    {
        "make_args": dict(n=n_sources, k=n_sources, n_orders=n_orders, stop_crit=stop_crit, max_iter=5, diffusion_parameter=diffusion_parameter),
        "solver_name": "SSM",
        "display_name": f"SSM",
        "prep_leadfield": prep_leadfield,
        "clim": {"kind": "value", "lims": [0., 0., 1]},
    },
    # {
    #     "make_args": dict(max_iter=200),
    #     "solver_name": "champagne-nl",
    #     "display_name": f"Champagne-NL",
    #     "prep_leadfield": True,
    #     "clim": {"kind": "value", "lims": [0., 0., 1]},
    # },
    # {
    #     "make_args": dict(max_iter=1000),
    #     "solver_name": "convexity champagne",
    #     "display_name": f"Convexity Champagne",
    #     "prep_leadfield": True,
    #     "clim": {"kind": "value", "lims": [0., 0., 1]},
    # }
]

C:\Users\lukas\AppData\Local\Temp\ipykernel_4356\1252984825.py:18: RuntimeWarning: 0.0% of original source space vertices have been omitted, tri-based adjacency will have holes.
Consider using distance-based adjacency or morphing data to all source space vertices.
  adjacency = mne.spatial_src_adjacency(src, verbose=0).toarray()


In [10]:
%matplotlib qt
from tqdm.notebook import tqdm
from statsmodels.graphics.gofplots import qqplot
import numpy as np
from funs import get_whitener, whiten_evoked_fwd
from scipy.sparse import csr_matrix
from time import time

get_idc = lambda long_list, short_list: np.array([long_list.index(elem) for elem in short_list])
distances = []
solvers = {solver_dict["display_name"]: None for solver_dict in solver_dicts}

event_id_list = list(event_ids.keys()) + [list(event_ids.keys())]
event_id_list = event_id_list[::-1]
event_id_dict = {(elem if type(elem)==str else "both"): elem for elem in event_id_list}


event_label = "both"
event_id = ['auditory_1', 'auditory_2']

noise_cov = mne.compute_covariance(epochs[event_id],
                                tmin=None, tmax=0,
                                    method="empirical", rank=None,
                                    verbose=0)

evokeds_w = dict()
evokeds = dict()
fwds_w = dict()
stcs = dict()
stcs_w = dict()
for time_label, (tmin, tmax) in time_frames.items():
    print(time_label)
    evokeds_w[time_label] = dict()
    evokeds[time_label] = dict()
    fwds_w[time_label] = dict()
    stcs[time_label] = dict()
    stcs_w[time_label] = dict()
    
    max_trials = len(epochs[event_id])
    
    # Average & Whiten Data
    evoked, evoked_w, fwd_w = whiten_evoked_fwd(epochs.copy(), fwd.copy(), noise_cov=noise_cov, tmin=None, tmax=0, method="empirical", verbose=0)
    
    # evoked_w.plot_joint(times=[-0.02, 0.089, 0.109, 0.129], title="Whitened Evoked")
    # evoked.plot_joint(times=[-0.02, 0.089, 0.109, 0.129], title="Non-Whitened Evoked")

    for solver_dict in solver_dicts:
        print("\t", solver_dict["display_name"])
        if solvers[solver_dict["display_name"]] is None:
            solver = Solver(solver_dict["solver_name"], n_reg_params=n_reg_params, prep_leadfield=solver_dict["prep_leadfield"])
            solvers[solver_dict["display_name"]] = solver
        
        
        solvers[solver_dict["display_name"]].make_inverse_operator(fwd_w.copy(), evoked_w.copy().crop(tmin, tmax), **solver_dict["make_args"])
        stc = solvers[solver_dict["display_name"]].apply_inverse_operator(evoked_w)
        
        while abs(stc.lh_data).max() == 0 or abs(stc.rh_data).max() == 0:
            print("\t\tMore sources!")
            solver_dict["make_args"]["n"] += 1
            solver_dict["make_args"]["k"] += 1
            solvers[solver_dict["display_name"]].make_inverse_operator(fwd_w.copy(), evoked_w.copy().crop(tmin, tmax), **solver_dict["make_args"])
            stc = solvers[solver_dict["display_name"]].apply_inverse_operator(evoked_w)
        

        stc.subject = subject
        stcs_w[time_label][solver_dict["display_name"]] = csr_matrix(stc.data)
        
        
        solvers[solver_dict["display_name"]].make_inverse_operator(fwd.copy(), evoked.copy().crop(tmin, tmax), **solver_dict["make_args"])
        stc = solvers[solver_dict["display_name"]].apply_inverse_operator(evoked)
        while abs(stc.lh_data).max() == 0 or abs(stc.rh_data).max() == 0:
            print("\t\tMore sources!")
            solver_dict["make_args"]["n"] += 1
            solver_dict["make_args"]["k"] += 1
            solvers[solver_dict["display_name"]].make_inverse_operator(fwd.copy(), evoked.copy().crop(tmin, tmax), **solver_dict["make_args"])
            stc = solvers[solver_dict["display_name"]].apply_inverse_operator(evoked)

        stc.subject = subject
        stcs[time_label][solver_dict["display_name"]] = csr_matrix(stc.data)
        
        # Plot
        stc.data /= np.max(abs(stc.data))
        # stc.plot(hemi="both", views="lat", cortex="low_contrast", subjects_dir=subjects_dir, time_viewer=True, brain_kwargs=dict(title=solver_dict["display_name"]))



    # Store    
    evokeds_w[time_label] = evoked_w
    evokeds[time_label] = evoked
    fwds_w[time_label] = fwd_w
    
    # QQ Plot
    # array = evokeds_w[time_label].copy().crop(tmin=None, tmax=0).data.flatten()
    # fig = qqplot(array,  line='45')

early
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Forward channels: 305
Epochs channels: 305


c:\Users\lukas\OneDrive\Dokumente\projects\flex_ssm\ssm_paper_analysis\funs.py:345: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in a change of behavior because the resulting channel order will not match. Either use a channel order that matches your instance or pass ordered=False.
  epochs = epochs.pick_channels(common_ch_names)


NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Removing projector <Projection | axial-10.0-200.0-PCA-01, active : True, n_channels : 102>
Removing projector <Projection | axial-10.0-200.0-PCA-02, active : True, n_channels : 102>
Removing projector <Projection | axial-10.0-200.0-PCA-03, active : True, n_channels : 102>
Removing projector <Projection | axial-10.0-200.0-PCA-04, active : True, n_channels : 102>
channels:  grad 203
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Removing projector <Projection | axial-10.0-200.0-PCA-01, active : True, n_channels : 102>
Removing projector <Projection | axial-10.0-200.0-PCA-02, active : True, n_channels : 102>
Removing projector <Projection | axial-10.0-200.0-PCA-03, active : True, n_channels : 102>
Removing projector <Projection | axial-10.0-200.0-PCA-04, active : True, n_channels : 102>
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
channels:  mag 102

c:\Users\lukas\OneDrive\Dokumente\projects\flex_ssm\ssm_paper_analysis\../../invert\invert\solvers\music.py:444: RuntimeWarning: 0.0% of original source space vertices have been omitted, tri-based adjacency will have holes.
Consider using distance-based adjacency or morphing data to all source space vertices.
  adjacency = mne.spatial_src_adjacency(self.forward['src'], verbose=0)


Source Iteration  0
Source Iteration  1
[[0, 1939], [0, 6187]]
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Source Iteration  0
Source Iteration  1
[[0, 1970], [0, 1849]]
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
		More sources!
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Source Iteration  0
Source Iteration  1
Source Iteration  2
[[0, 1970], [0, 1849], [0, 5934]]
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should

...

NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
		More sources!
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
late
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Forward channels: 305
Epochs channels: 305


c:\Users\lukas\OneDrive\Dokumente\projects\flex_ssm\ssm_paper_analysis\funs.py:345: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in a change of behavior because the resulting channel order will not match. Either use a channel order that matches your instance or pass ordered=False.
  epochs = epochs.pick_channels(common_ch_names)


NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Removing projector <Projection | axial-10.0-200.0-PCA-01, active : True, n_channels : 102>
Removing projector <Projection | axial-10.0-200.0-PCA-02, active : True, n_channels : 102>
Removing projector <Projection | axial-10.0-200.0-PCA-03, active : True, n_channels : 102>
Removing projector <Projection | axial-10.0-200.0-PCA-04, active : True, n_channels : 102>
channels:  grad 203
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Removing projector <Projection | axial-10.0-200.0-PCA-01, active : True, n_channels : 102>
Removing projector <Projection | axial-10.0-200.0-PCA-02, active : True, n_channels : 102>
Removing projector <Projection | axial-10.0-200.0-PCA-03, active : True, n_channels : 102>
Removing projector <Projection | axial-10.0-200.0-PCA-04, active : True, n_channels : 102>
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
channels:  mag 102

c:\Users\lukas\OneDrive\Dokumente\projects\flex_ssm\ssm_paper_analysis\funs.py:345: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in a change of behavior because the resulting channel order will not match. Either use a channel order that matches your instance or pass ordered=False.
  epochs = epochs.pick_channels(common_ch_names)


NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Removing projector <Projection | axial-10.0-200.0-PCA-01, active : True, n_channels : 102>
Removing projector <Projection | axial-10.0-200.0-PCA-02, active : True, n_channels : 102>
Removing projector <Projection | axial-10.0-200.0-PCA-03, active : True, n_channels : 102>
Removing projector <Projection | axial-10.0-200.0-PCA-04, active : True, n_channels : 102>
channels:  grad 203
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Removing projector <Projection | axial-10.0-200.0-PCA-01, active : True, n_channels : 102>
Removing projector <Projection | axial-10.0-200.0-PCA-02, active : True, n_channels : 102>
Removing projector <Projection | axial-10.0-200.0-PCA-03, active : True, n_channels : 102>
Removing projector <Projection | axial-10.0-200.0-PCA-04, active : True, n_channels : 102>
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
channels:  mag 102

## Load

In [ ]:
import pickle as pkl
fns  =[
    "stcs_new_cov_late_500_kfSLDZUayS.pkl",
]
stc_list = []
for fn in fns:
    with open(fn, "rb") as file:
        stcs = pkl.load(file)
        stc_list.append(stcs)

# Merge the nested stc dicts:
stcs = stc_list[0]
for i_next in range(1, len(stc_list)):
    print(i_next)
    for ntr in stc_list[i_next]["both"].keys():
        n = list(stcs["both"][ntr].keys())[-1]
        new_dict = {key+n+1: value  for key, value in stc_list[i_next]["both"][ntr].items()}
        if not ntr in stcs["both"].keys():
            stcs["both"][ntr] = new_dict
        else:
            stcs["both"][ntr].update(new_dict)

In [ ]:
# evokeds_w["auditory_1"][n_trials[-1]][0].plot_joint()
# evokeds_w["auditory_2"][n_trials[-1]][0].plot_joint()
# evokeds_w["both"][n_trials[-1]][0].plot_joint()
from copy import deepcopy
stc_ = stc.copy()
stc_.data = deepcopy(stcs["both"][n_trials[-1]][0]["AP"].toarray())
stc_.data /= np.max(abs(stc_.data))
stc_.plot(surface="inflated", hemi="both", cortex="low_contrast", subjects_dir=subjects_dir, time_viewer=True, initial_time=0.109, time_unit="s")
                     

# mne-python inverse

In [ ]:
from mne.minimum_norm import make_inverse_operator, apply_inverse

ntr = n_trials[-1]
snr = 3
lambda2 = 1.0 / snr**2
method = "dSPM"
event_id = "both"

inverse_operator = make_inverse_operator(
    evokeds[event_id][ntr][0].info, fwd, noise_covs[event_id], fixed=True, depth=None
    )

# Compute inverse solution on contrast
stc = apply_inverse(evokeds[event_id][ntr][0], inverse_operator, lambda2, method, pick_ori=None)
# stc.save('spm_%s_dSPM_inverse' % contrast.comment)
stc.data /= abs(stc.data).max()
# Plot contrast in 3D with mne.viz.Brain if available
brain = abs(stc).plot(
    hemi="both",
    subjects_dir=subjects_dir,
    initial_time=0.119,
    views=["lat"],
    clim={"kind": "value", "pos_lims": [0., 0.5, 1]},
    cortex="low_contrast"
)



# Atlas label
atlas = "aparc.a2009s"
labels_lh = mne.read_labels_from_annot(
    subject, atlas, "lh", subjects_dir=subjects_dir
)
labels_rh = mne.read_labels_from_annot(
    subject, atlas, "rh", subjects_dir=subjects_dir
)
labels = labels_lh + labels_rh
labels
label_heschl = [label for label in labels if "S_temporal_transverse" in label.name]
label_heschl

for l in label_heschl:
    brain.add_label(l, borders=True, color="black")
# brain.save_image(f'{save_path}/{method}_map.png')

# Establish Proxy of Ground Truth

In [11]:
%matplotlib qt
import sys; sys.path.insert(0, r'C:\Users\Lukas\OneDrive\Documents\projects\invert')
from invert import Solver
from invert.util import pos_from_forward
n_chans, n_dipoles = fwd["sol"]["data"].shape
# get left and right channel names
evoked = evokeds_w[list(time_frames.keys())[0]]

ch_names = evoked.ch_names
ch_names_left = [ch["ch_name"] for ch in evoked.info["chs"] if ch["loc"][0]<0]
ch_names_right = [ch["ch_name"] for ch in evoked.info["chs"] if ch["loc"][0]>=0]

n_orders = 0
n_sources = 1
prep_leadfield = False
gt_solver_names = ["RAP-MUSIC", "AP", "SSM"]
ground_truth_idc = dict()

for gt_solver_name in gt_solver_names:
    ground_truth_idc[gt_solver_name] = []

    for i_tf, (time_frame, (tmin, tmax)) in enumerate(time_frames.items()):
        print(time_frame)
        evoked = evokeds_w[time_frame]
        # evoked = evokeds[time_frame]

        evoked_sub = dict()
        evoked_sub["lh"] = evoked.copy().pick_channels(ch_names_left)
        evoked_sub["rh"] = evoked.copy().pick_channels(ch_names_right)
        ground_truth_idc[gt_solver_name].append([])


        for hem in ("lh", "rh"):  # left vs right hemisphere
            fwd_temp = fwds_w[time_frame].copy().pick_channels(evoked_sub[hem].ch_names)
            # Left dipole:
            solver = Solver(gt_solver_name, prep_leadfield=prep_leadfield)
            solver.make_inverse_operator(fwd_temp, evoked_sub[hem].copy().crop(tmin=tmin, tmax=tmax), n=n_sources, k=n_sources, n_orders=n_orders, stop_crit=0)
            stc = solver.apply_inverse_operator(evoked_sub[hem])
            ground_truth_idc[gt_solver_name][i_tf].append(np.argmax(abs(stc.data[:, 0])))

            stc.subject = subject
            stc.data /= abs(stc.data).max()
            brain = stc.plot(brain_kwargs=dict(title=f"{solver.name} {tmin} - {tmax} stim, {hem}"), surface="inflated", hemi="both", subjects_dir=subjects_dir, subject=subject, cortex="low_contrast", time_viewer=False, verbose=0)
            if hem=="lh":
                brain.add_foci(stc.lh_vertno[ground_truth_idc[gt_solver_name][i_tf][-1]], color="green", coords_as_verts=True, hemi="lh")
            else:
                brain.add_foci(stc.rh_vertno[ground_truth_idc[gt_solver_name][i_tf][-1] - int(n_dipoles / 2)], color="green", coords_as_verts=True, hemi="rh")
        
        source_data = np.zeros((n_dipoles, 1))
        source_data[ground_truth_idc[gt_solver_name][i_tf][0]] = 1
        source_data[ground_truth_idc[gt_solver_name][i_tf][1]] = 1
        stc.data = source_data
        stcs_w[time_frame][f"Ground Truth ({gt_solver_name})"] = csr_matrix(stc.data)
            

early
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


c:\Users\lukas\OneDrive\Dokumente\projects\flex_ssm\ssm_paper_analysis\../../invert\invert\solvers\music.py:444: RuntimeWarning: 0.0% of original source space vertices have been omitted, tri-based adjacency will have holes.
Consider using distance-based adjacency or morphing data to all source space vertices.
  adjacency = mne.spatial_src_adjacency(self.forward['src'], verbose=0)


Source Iteration  0
[[0, 1939]]
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


C:\Users\lukas\AppData\Local\Temp\ipykernel_4356\3289204714.py:43: RuntimeWarning: All data were zero
  brain = stc.plot(brain_kwargs=dict(title=f"{solver.name} {tmin} - {tmax} stim, {hem}"), surface="inflated", hemi="both", subjects_dir=subjects_dir, subject=subject, cortex="low_contrast", time_viewer=False, verbose=0)


NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


...

C:\Users\lukas\AppData\Local\Temp\ipykernel_4356\3289204714.py:43: RuntimeWarning: All data were zero
  brain = stc.plot(brain_kwargs=dict(title=f"{solver.name} {tmin} - {tmax} stim, {hem}"), surface="inflated", hemi="both", subjects_dir=subjects_dir, subject=subject, cortex="low_contrast", time_viewer=False, verbose=0)


NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


c:\Users\lukas\OneDrive\Dokumente\projects\flex_ssm\ssm_paper_analysis\../../invert\invert\solvers\music.py:1123: RuntimeWarning: 0.0% of original source space vertices have been omitted, tri-based adjacency will have holes.
Consider using distance-based adjacency or morphing data to all source space vertices.
  adjacency = mne.spatial_src_adjacency(self.forward['src'], verbose=0)


NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


C:\Users\lukas\AppData\Local\Temp\ipykernel_4356\3289204714.py:43: RuntimeWarning: All data were zero
  brain = stc.plot(brain_kwargs=dict(title=f"{solver.name} {tmin} - {tmax} stim, {hem}"), surface="inflated", hemi="both", subjects_dir=subjects_dir, subject=subject, cortex="low_contrast", time_viewer=False, verbose=0)


# Compute MLEs for each algo

In [16]:
from scipy.optimize import linear_sum_assignment
pp = dict(hemi="both", verbose=0, cortex="low_contrast", surface="inflated", subject=subject, subjects_dir=subjects_dir)
n_chans, n_dipoles = fwd["sol"]["data"].shape
mles = []
for gt_solver_name, ground_truth_idc_solver in ground_truth_idc.items():
    for i_tf, time_frame in enumerate(time_frames.keys()):
        true_idc = ground_truth_idc_solver[i_tf]
        data = np.zeros((n_dipoles, 1))
        data[true_idc] = 1
        stc.data = data
        # stc.plot(brain_kwargs=dict(title=f"Ground Truth {time_frame}"), **pp)
        
        for algo, source_data in stcs[time_frame].items():
        # for algo, source_data in stcs_w[time_frame].items():
            if algo == "Ground Truth":
                continue
            est_idc = np.where(source_data.toarray()[:, 0] != 0)[0]
            print(est_idc)
    	    
            pairwise_dist = cdist(pos[true_idc], pos[est_idc])
            # Solve the assignment problem (i.e., find the matching with minimum total distance)
            true_indices, estimated_indices = linear_sum_assignment(pairwise_dist)
            # Calculate the sum of distances for the optimal assignment
            mle = pairwise_dist[true_indices, estimated_indices].mean()
            print("\t", algo, ": ", mle)
            mles.append(dict(mle=mle, time_frame=time_frame, algo=algo, ground_truth=gt_solver_name, idc=est_idc[estimated_indices], gt_idc=np.array(true_idc)[true_indices]))

            data = np.zeros((n_dipoles, 1))
            data[est_idc[estimated_indices]] = 1
            stc.data = data
            # stc.plot(brain_kwargs=dict(title=f"{algo} {time_frame}"), **pp)

[1849 1970 5934]
	 RAP-MUSIC :  10.619330883602592
[1940 6141]
	 AP :  10.420954566320884
[1701 1977 5934]
	 SSM :  9.766532720949265
[1701 1849 5853]
	 RAP-MUSIC :  12.208125531484047
[1888 6141]
	 AP :  10.979126177137326
[1701 1849 5850]
	 SSM :  13.121698516022205
[1701 1849 5855]
	 RAP-MUSIC :  16.083422287498145
[1977 6141]
	 AP :  7.807877930021009
[1701 1888 5697]
	 SSM :  13.406158677772057
[1849 1970 5934]
	 RAP-MUSIC :  10.619330883602592
[1940 6141]
	 AP :  10.420954566320884
[1701 1977 5934]
	 SSM :  9.766532720949265
[1701 1849 5853]
	 RAP-MUSIC :  12.208125531484047
[1888 6141]
	 AP :  10.979126177137326
[1701 1849 5850]
	 SSM :  13.121698516022205
[1701 1849 5855]
	 RAP-MUSIC :  16.315651801388533
[1977 6141]
	 AP :  8.768161631991415
[1701 1888 5697]
	 SSM :  15.353884327838495
[1849 1970 5934]
	 RAP-MUSIC :  8.341644542939404
[1940 6141]
	 AP :  9.265637542831776
[1701 1977 5934]
	 SSM :  7.488846380286077
[1701 1849 5853]
	 RAP-MUSIC :  10.099466462589918
[1888 6141]

In [17]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Sample data in DataFrame 'df'
df = pd.DataFrame(mles)

# Initialize a figure with 3 subplots (horizontally)
fig, axes = plt.subplots(1, 4, figsize=(18, 6), sharey=True)

# Iterate over each unique ground truth basis and plot on a subplot
for i, ground_truth_basis in enumerate(gt_solver_names):
    df_ = df[df.ground_truth == ground_truth_basis]
    df_ = df_[~df_.algo.str.contains("Ground Truth")]
    # Use 'axes[i]' to specify which subplot to use
    sns.barplot(data=df_, y="mle", x="time_frame", hue="algo", ax=axes[i])
    axes[i].set_title(f"{ground_truth_basis}-based")
    axes[i].set_xlabel("Time Frame")  # Set x-axis label
    axes[i].set_ylabel("Mean Localization Error [mm]")  # Set y-axis label


df_match = df.copy()
df_match["match"] = df_match["algo"] == df_match["ground_truth"]
df_match = df_match[df_match["match"]]
sns.barplot(data=df_match, y="mle", x="time_frame", hue="algo", ax=axes[-1])
axes[-1].set_title("Individual Ground Truth ")
axes[-1].set_ylabel("Mean Localization Error [mm]")  # Set y-axis label
axes[-1].set_xlabel("Time Frame")  # Set x-axis label

# Display the legend only on the last subplot
axes[-1].legend(loc='upper right')
# Remove the legend from other subplots
for ax in axes[:-1]:
    ax.get_legend().remove()
plt.suptitle("Ground Truth Trick with Different Solvers", fontsize=16)  # Set the title of the figure
plt.tight_layout()
plt.show()
# save 
plt.savefig(f"figures/auditory_dimitrios_new/ground_truth_trick_{sampling}_non-whitened.png")

# Create Source Plots

In [18]:
def trim(img):
    left = np.where((img-255).sum(axis=(0,-1))!=0)[0][0]
    right = np.where((img-255).sum(axis=(0,-1))!=0)[0][-1]

    upper = np.where((img-255).sum(axis=(1,-1))!=0)[0][0]
    lower = np.where((img-255).sum(axis=(1,-1))!=0)[0][-1]
    img = img[upper:lower, left:right]

    return img

def pad(imgs, pad_value=255):
    image1, image2 = imgs
    # Get the shapes of both images
    shape1 = image1.shape
    shape2 = image2.shape

    # Calculate padding for height and width
    pad_height = max(shape1[0], shape2[0]) - np.array([shape1[0], shape2[0]])
    pad_width = max(shape1[1], shape2[1]) - np.array([shape1[1], shape2[1]])

    # Pad both images using the specified pad_value
    padded_image1 = np.pad(image1, ((0, pad_height[0]), (0, pad_width[0]), (0,0)), mode='constant', constant_values=pad_value)
    padded_image2 = np.pad(image2, ((0, pad_height[1]), (0, pad_width[1]), (0,0)), mode='constant', constant_values=pad_value)

    return padded_image1, padded_image2

In [19]:
pp = dict(surface="inflated", verbose=0, cortex="low_contrast", subject=subject, subjects_dir=subjects_dir, colorbar=False, background="white", time_viewer=False, clim=dict(kind="value", lims=(0, 0.1, 1)))
hemis = ["rh", "lh"]
view = "lateral"
algo_order = ["SSM", "AP", "RAP-MUSIC"]
gt_order = algo_order
views = ()
for time_frame, (tmin, tmax) in time_frames.items():
    fig, axes = plt.subplots(3, 2, figsize=(12, 6))
    plt.suptitle(f"{time_frame.capitalize()} Time Frame ({round(tmin*1000)} - {round(tmax*1000)} ms)")

    for i_row, algo in enumerate(algo_order):

        ax = axes[i_row, 0]
        ax.axis('off')

        df_ = df[df.ground_truth==algo]
        df_ = df_[df_.time_frame==time_frame]
        idc = df_.gt_idc.values[0]
        source_data = np.zeros((n_dipoles, 1))
        source_data[idc] = 1
        stc.data = source_data
        imgs = []
        for hemi in hemis:
            pp["views"] = view
            pp["hemi"] = hemi
            brain = stc.plot(**pp)
            img = brain.screenshot()
            brain.close()
            img = trim(img)
            imgs.append(img)
        img = np.concatenate(pad(imgs), axis=1)

        ax.imshow(img)
        ax.set_title(f"Ground Truth ({algo})")


        ax = axes[i_row, 1]
        ax.axis('off')

        df_ = df[df.algo==algo]
        df_ = df_[df_.time_frame==time_frame]
        idc = df_.idc.values[0]
        source_data = np.zeros((n_dipoles, 1))
        source_data[idc] = 1
        stc.data = source_data
        imgs = []
        for hemi in hemis:
            pp["views"] = view
            pp["hemi"] = hemi
            brain = stc.plot(**pp)
            img = brain.screenshot()
            brain.close()
            img = trim(img)
            imgs.append(img)
        img = np.concatenate(pad(imgs), axis=1)

        ax.imshow(img)
        ax.set_title(f"{algo}")
    plt.savefig(f"figures/auditory_dimitrios_new/brain_plots_{time_frame}_{sampling}_non-whitened.png")

        

In [31]:
# plt.figure()
# plt.imshow(noise_cov.data)

# plt.colorbar()

# Extract the covariance matrix data
cov_matrix = noise_cov.data

# Convert covariance matrix to correlation matrix
diag = np.sqrt(np.diag(cov_matrix))
corr_matrix = cov_matrix / np.outer(diag, diag)

plt.figure(figsize=(8, 13))
plt.subplot(211)
plt.imshow(corr_matrix, vmin=-1, vmax=1)
plt.colorbar()
plt.title("Noise Covariance Matrix")

plt.subplot(212)
sns.distplot(abs(corr_matrix.flatten()))
plt.title(f"Distribution of pearson correlation coefficients between channels\nMedian: {np.median(corr_matrix):.2f}, Mean: {np.mean(corr_matrix):.2f}")

plt.tight_layout()

plt.savefig(f"figures/auditory_dimitrios_new/noise_covariance.png")

C:\Users\lukas\AppData\Local\Temp\ipykernel_4356\2879993278.py:20: UserWarning: 

`distplot` is a deprecated function and will be removed in seaborn v0.14.0.

Please adapt your code to use either `displot` (a figure-level function with
similar flexibility) or `histplot` (an axes-level function for histograms).

For a guide to updating your code to use the new functions, please see
https://gist.github.com/mwaskom/de44147ed2974457ad6372750bbe5751

  sns.distplot(abs(corr_matrix.flatten()))


# Other

In [ ]:
tmin, tmax = 0.109, 0.129
fwd = fwds_w["both"][345][0]
evoked = evokeds_w["both"][345][0]
for solver_dict in solver_dicts:
    solver_name = solver_dict["solver_name"]
    display_name = solver_dict["display_name"]
    make_args = solver_dict["make_args"]
    clim = solver_dict["clim"]
    display_name = solver_dict["display_name"]
    solver = Solver(solver_name)
    solver.make_inverse_operator(fwd, evoked.copy().crop(tmin, tmax), **make_args)
    stc = solver.apply_inverse_operator(evoked.copy())
    stc.subject = subject
    brain = stc.plot(
        hemi="both",
        subjects_dir=subjects_dir,
        initial_time=0.119,
        views=["lat"],
        clim={"kind": "percent", "lims": [0, 50, 100]},
        brain_kwargs=dict(title=f"{solver.name}-{n_orders}-{n_sources}-{diffusion_parameter}-({tmin}-{tmax}s)"),
        # surface="pial",
        surface="inflated",
        cortex="low_contrast"
        )
    brain.add_foci(stc.lh_vertno[ground_truth_idc[-1]], color="green", coords_as_verts=True, hemi="lh")
    brain.add_foci(stc.rh_vertno[ground_truth_idc[-1] - int(8194 / 2)], color="green", coords_as_verts=True, hemi="rh")

In [ ]:
from scipy.spatial.distance import cdist
pos_left = mne.vertex_to_mni(fwd["src"][0]["vertno"], 0, subject=subject, subjects_dir=subjects_dir, verbose=0)
pos_right = mne.vertex_to_mni(fwd["src"][1]["vertno"], 1, subject=subject, subjects_dir=subjects_dir, verbose=0)
pos = np.concatenate([pos_left, pos_right], axis=0)

pos_idc_left = pos[:, 0] <= 0
pos_idc_right = pos[:, 0] > 0

n_sources = 2

stcs = dict()
localization_errors = []

for side in ("both",):  # "right"):
    print(side)
    maxima_true = np.stack([
        pos[ground_truth_idc[0], :],
        pos[ground_truth_idc[1], :],
        ], axis=0)
    stcs[side] = dict()
    
    for tr in n_trials:#[-1:]:
        print("\t", tr)
        stcs[side][tr] = dict()
        evoked_list = evokeds_w[side][tr]
        fwd = fwds_w[side][tr][0]
        
        for solver_dict in solver_dicts:
            if not "-" in solver_dict["name"]:
                continue
            print("\t\t", solver_dict["name"])
            stcs[side][tr][solver_dict["show_name"]] = []
            for i_sample, evoked in enumerate(evoked_list):
                print("\t\t\t", f"sample {i_sample}")

                solver = Solver(solver_dict["name"], prep_leadfield=solver_dict["prep_leadfield"])
                solver.make_inverse_operator(fwd.copy(), evoked.copy().crop(tmin, tmax), **solver_dict["make_args"])
                stc = solver.apply_inverse_operator(evoked.copy())
                
                has_dipole_left = sum(stc.data[pos_idc_left, 0] != 0) > 0
                has_dipole_right = sum(stc.data[pos_idc_right, 0] != 0) > 0
                
                while not (has_dipole_left and has_dipole_right):
                    solver_dict["make_args"]["n"] += 1
                    solver_dict["make_args"]["k"] += 1
                    solver.make_inverse_operator(fwd.copy(), evoked.copy().crop(tmin, tmax), **solver_dict["make_args"])
                    stc = solver.apply_inverse_operator(evoked.copy())

                    has_dipole_left = sum(stc.data[pos_idc_left, 0] != 0) > 0
                    has_dipole_right = sum(stc.data[pos_idc_right, 0] != 0) > 0
                    print("\t\t\tincreasing n_sources by one")
                solver_dict["make_args"]["n"] = n_sources
                solver_dict["make_args"]["k"] = n_sources
                
                

                stc.subject = subject
                stcs[side][tr][solver_dict["show_name"]].append(stc)
                
                # brain = stc.plot(brain_kwargs=dict(title=f"{solver.name} {side} stim, {tr} trials"), surface="inflated", hemi="both", subjects_dir="C:\\Users\\lukas\\mne_data\\MNE-sample-data\\subjects", subject="sample", verbose=0)
                
                # brain.add_foci(stc.lh_vertno[ground_truth_idc[side][0]], coords_as_verts=True, color="green", hemi="lh", scale_factor=1)
                # brain.add_foci( stc.rh_vertno[ground_truth_idc[side][1]-len(stc.lh_vertno)], coords_as_verts=True, color="green", hemi="rh", scale_factor=1)

                stc_lh = stc.copy()
                stc_rh = stc.copy()
                
                stc_lh.data[pos_idc_right, :] = 0
                stc_rh.data[pos_idc_left, :] = 0
                
                
                maximum_idx_lh = np.argmax(abs(stc_lh.data).mean(axis=1))
                maximum_idx_rh = np.argmax(abs(stc_rh.data).mean(axis=1))

                maximum_lh, maximum_rh = pos[maximum_idx_lh], pos[maximum_idx_rh]
                maxima_est = np.stack([maximum_lh, maximum_rh], axis=0)
                

                distmat = cdist(maxima_true, maxima_est)
                mle = (np.mean(distmat.min(axis=0)) + np.mean(distmat.min(axis=1))) / 2
                localization_errors.append(
                    dict(mle=mle, side=side, trials=tr, solver_name=solver_dict["show_name"], sample=i_sample)
                )
                # print(distmat)
                # break

        # break
    # break
            

# plot mu

In [ ]:
stc.data = solver.mu.T
stc.data /= abs(stc.data).max()
stc.subject = subject
brain = abs(stc).plot(
    hemi="both",
    subjects_dir=subjects_dir,
    initial_time=0.12,
    views=["lat"],
    clim={"kind": "percent", "lims": [0, 50, 100]},
    brain_kwargs=dict(title=f"{solver.name}-{n_orders}-{n_sources}-{diffusion_parameter}-({tmin}-{tmax}s)"),
    # cortex="low_contrast",
    # surface="pial"
    surface="inflated"
)

# Atlas label
atlas = "aparc.a2009s"
labels_lh = mne.read_labels_from_annot(
    subject, atlas, "lh", subjects_dir=subjects_dir
)
labels_rh = mne.read_labels_from_annot(
    subject, atlas, "rh", subjects_dir=subjects_dir
)
labels = labels_lh + labels_rh
labels
label_heschl = [label for label in labels if "S_temporal_transverse" in label.name]
label_heschl

for l in label_heschl:
    brain.add_label(l, borders=True, color="black")

# Screen for optimal time range

In [ ]:
from invert.evaluate import get_maxima
from scipy.spatial.distance import cdist
event_id = "auditory_1"

adjacency = mne.spatial_src_adjacency(fwds[event_id]['src'], verbose=0).toarray()
pos_left = mne.vertex_to_mni(fwd["src"][0]["vertno"], 0, subject=subject, subjects_dir=subjects_dir, verbose=0)
pos_right = mne.vertex_to_mni(fwd["src"][1]["vertno"], 1, subject=subject, subjects_dir=subjects_dir, verbose=0)
pos = np.concatenate([pos_left, pos_right], axis=0)

fwd_w = fwds[event_id]
evoked_w = evokeds_w[event_id]

n_sources = 2
n_orders = 10
diffusion_parameter = 0.1

target_pos = np.array([
    [55.6, -20.3, 4.8],
    [-50.5, -24.9, 4.2]
    ])
tmins = np.arange(0.04, 0.2, 0.01)
durations = np.array([0.01, 0.02, 0.04])
diffusion_parameters = [0.1,]
print(len(tmins)*len(durations)*len(diffusion_parameters), " iterations")
solver_name = "AP"
solver = Solver(solver_name, n_reg_params=25, prep_leadfield=False)
results = []
for diffusion_parameter in diffusion_parameters:
    for tmin in tmins:
        for duration in durations:
            tmax = tmin + duration

            solver.make_inverse_operator(fwd_w, evoked_w.copy().crop(tmin, tmax), n=n_sources, k=n_sources, n_orders=n_orders, stop_crit=0, max_iter=1000, diffusion_parameter=diffusion_parameter)
            stc = solver.apply_inverse_operator(evoked_w)
            vec = np.mean(abs(stc.copy().crop(tmin, tmax).data), axis=1)

            maxima_est = pos[np.array(get_maxima(vec, adjacency)), :]
            d = cdist(target_pos, maxima_est)
            mle = (d.min(axis=1).mean() + d.min(axis=0).mean()) / 2
            d = dict(diffusion_parameter=diffusion_parameter, tmin=tmin, duration=duration, n_sources=n_sources, n_orders=n_orders, mle=mle, stc=stc)
            results.append(d)
            print("\n\t", d, "\n")
            

In [ ]:
import pandas as pd
import seaborn as sns

df = pd.DataFrame(results)
plt.figure()
sns.barplot(x="tmin", hue="diffusion_parameter", y="mle", data=df)

In [ ]:
for i in range(10):
    stc = df.sort_values("mle")["stc"].values[i].copy()
    stc.subject = subject
    stc.data /= abs(stc.data).max()
    stc.plot(
        surface="inflated",
        hemi="both",
        subjects_dir=subjects_dir,
        initial_time=0.12,
        views=["lat"],
        clim={"kind": "percent", "lims": [0, 50, 100]},
        brain_kwargs=dict(title=f"{solver.name}-{n_orders}-{n_sources}-{diffusion_parameter}-({tmin}-{tmax}s)")
    )
    # break


In [ ]:
np.where(stc.data[:, 0]!=0)[0]

# werked:

In [ ]:
idx diffusion_parameter	tmin	duration	n_sources	n_orders	mle	stc
5	0.1	0.11	0.04	2	10	15.368256	<SourceEstimate | 8diffusion_parameter	tmin	duration	n_sources	n_orders	mle	stc194 vertices, subject : tas...
7	0.1	0.12	0.02	2	10	15.368256	<SourceEstimate | 8194 vertices, subject : tas...